In [1]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd

In [2]:
ref_keys = pickle.load(open("pickles_data/references_keys.pkl", "rb"))

In [3]:
author_level = pickle.load(open("pickles/paper_info (1).pickle", "rb"))

In [3]:
aid_list = pickle.load(open("aid_list.pickle", "rb"))

In [5]:
print(len(aid_list))

91137597


In [6]:
mapped_arnet_ids_to_acl = []
id_to_name = {}
name_to_id = {}
for paper , citations in author_level.items():
    for citation in citations:
        arnet_id = citation['arnet_id']
        if arnet_id not in id_to_name :
            id_to_name[arnet_id] = []
        if(citation['paper_name'] not in id_to_name[arnet_id]):
            id_to_name[arnet_id].append(citation['paper_name'])
        name_to_id[citation['paper_name']] = arnet_id
        mapped_arnet_ids_to_acl.append(arnet_id)

In [7]:
len(mapped_arnet_ids_to_acl)

29973

In [8]:
mapped_micro_ids = list(set(mapped_arnet_ids_to_acl).intersection(set(aid_list)))

In [9]:
print(len(mapped_micro_ids))

9604


In [43]:
mapped_arnet_ids = list(set(mapped_arnet_ids_to_acl).intersection(set(ref_keys)))

In [44]:
count = 0 
non_uniq_arnet_ids = []
for key in mapped_arnet_ids_to_acl :
    if(key in mapped_arnet_ids) :
        non_uniq_arnet_ids.append(key)
        count+=1
print(count)

25083


In [45]:
len(list(set(mapped_arnet_ids_to_acl)))

13056

In [46]:
len(mapped_arnet_ids)

10564

In [47]:
pickle.dump(id_to_name, open("pickles_data/arnetids_to_papername.pkl", "wb"))

In [22]:
f = open("pickles_data/total_titles.txt")

In [23]:
data = []
for line in f :
    data.append(line)

In [25]:
titles = pickle.load(open("pickles_data/citation_titles.pkl", "rb"))

In [26]:
len(titles)

1181

In [10]:
def get_citations(folder):
    citation_list = {}
    for file in os.listdir(folder) :
        tree = ET.parse(folder+file)
        root = tree.getroot()
        id = file[:8]
        for element in root.iterfind("algorithm"):
            if(element.attrib['name']=="ParsCit"):
                citlist = element.getchildren()
                cits = citlist[0].getchildren()
                citations = []
                for cit in cits:
                    cit_dict = {}
                    if(cit.attrib['valid']=="true"):
                        try :
                            title = cit.find('title').text.lower()
                        except :
                            title = cit.find('rawString').text.lower()
                        cit_dict['title'] = title
                        cit_dict['cit'] = cit
                        citations.append(cit_dict)
                
                citation_list[id] = citations
            
    return citation_list 

In [11]:
citations = get_citations("xmls/")

In [12]:
titles = []
for id in citations :
    papers = citations[id]
    for paper in papers :
        titles.append(paper['title'])

In [13]:
len(titles)

31943

In [15]:
len(non_uniq_arnet_ids)

25083

In [48]:
arnet_names_v1 = []
for id in mapped_arnet_ids :
    arnet_names_v1.extend(id_to_name[id])

In [15]:
micro_names = []
for id in mapped_micro_ids :
    micro_names.extend(id_to_name[id])

In [17]:
print(len(micro_names))

11244


In [61]:
count_mapped = 0
v1_mapped = []
for tit in titles :
    if tit in arnet_names_v1 :
        v1_mapped.append(tit)
        count_mapped+=1

print(count_mapped)
print(len(v1_mapped))
print(len(set(v1_mapped)))

25113
25113
12399


In [19]:
count_mapped = 0
micro_mapped = []
for tit in titles :
    if tit in micro_names :
        micro_mapped.append(tit)
        count_mapped+=1

print(count_mapped)
print(len(micro_mapped))
print(len(set(micro_mapped)))

21595
21595
11240


In [55]:
arnet_names = list(name_to_id.keys())

In [62]:
count_mapped = 0
v2_mapped = []
for tit in titles :
    if tit in arnet_names :
        v2_mapped.append(tit)
        count_mapped+=1

print(count_mapped)
print(len(v2_mapped))
print(len(set(v2_mapped)))

30006
30006
15261


In [19]:
print(len(name_to_id.keys()))

15261


In [20]:
arnet_name_id = []
for name in name_to_id :
    arnet_name_id.append(name_to_id[name])

In [23]:
non_uniq_keys = []
for id in arnet_name_id :
    if(arnet_name_id.count(id)>1) :
        non_uniq_keys.append(id)

In [35]:
titles_arnet = []
for id in non_uniq_arnet_ids :
    titles_arnet.append(id_to_name[id])

In [20]:
tags = pickle.load(open("pickles_data/baseline_tags.pkl", "rb"))

In [21]:
baselines = []
for id in tags :
    for paper in tags[id] :
        if(paper['tag']==1) :
            baselines.append(paper['paper_name'])

In [22]:
print(len(baselines))

2953


In [54]:
count = 0
for title in baselines :
    if title in arnet_names_v1 :
        count+=1

print(count)

2484


In [23]:
count = 0
for title in baselines :
    if title in micro_names :
        count+=1

print(count)

2145


In [57]:
count = 0
for title in baselines :
    if title in arnet_names :
        count+=1

print(count)

2860


In [58]:
2953-2484

469

In [42]:
print(len(list(set(titles))))

16944


In [43]:
titles_arnet_v2 = []
for id in mapped_arnet_ids_to_acl :
    titles_arnet_v2.append(id_to_name[id])

In [44]:
len(titles_arnet_v2)

29973

In [59]:
count = 0
list_base = []
for title in baselines :
    if(title in arnet_names_v1) :
        list_base.append(title)
        count+=1
print(count)

2484


In [60]:
not_present = []
for base in baselines :
    if(base not in list_base) :
        not_present.append(base)
        
print(len(not_present))
print(len(set(not_present)))

469
352


In [47]:
print(len(set(list_base)))

1774


In [48]:
print(len(set(baselines)))

2178
